In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F # for activation functions. On the other hand, for adding act. func. we can use torch.nn as well.
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.tensorboard import SummaryWriter

In [12]:
class CNN(nn.Module):
    def __init__(self, in_channels=1, num_classes=10):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=8, kernel_size=(3,3), stride=(1,1), padding=(1,1)) # values has been chosen arbitrarily.
        self.pool = nn.MaxPool2d(kernel_size=(2,2), stride=(2,2))
        self.conv2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=(3,3), stride=(1,1), padding=(1,1))
        self.fc1 = nn.Linear(16*7*7, num_classes) # 16 => output channel of conv2 layer, 7*7 means after two pooling layer with 2x2 kernel, 28x28 input will be decrease to 7x7.

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)

        x = x.reshape(x.shape[0], -1)
        x = self.fc1(x)

        return x

def save_checkpoint(state, filename="./my_checkpoint.pt"):
    print("Saving Checkpoint")
    torch.save(state, filename)

def load_checkpoint(checkpoint):
    print("Loading Checkpoint")
    print(checkpoint.keys())
    model.load_state_dict(checkpoint['state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer'])


model = CNN()
x = torch.rand(64,1,28,28)
print(model(x).shape)

torch.Size([64, 10])


In [13]:
# device setup

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [14]:
# hyperparameters

in_channel = 1
num_classes = 10
learning_rate = 0.001
batch_size = 64
num_epochs = 10
load_model = True

In [15]:
# LOAD DATA

train_dataset = datasets.MNIST(root='dataset/', train=True, transform=transforms.ToTensor(), download=True) # ToTensor() converts original data that loaded from dataset library as np.array, to tensor. 
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True) # Shuffle makes us sure about do not have same images on batches on different epochs.

test_dataset = datasets.MNIST(root='dataset/', train=False, transform=transforms.ToTensor(), download=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

In [16]:
# INIT MODEl
model = CNN().to(device) # send to GPU if you have CUDA.  

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
writer = SummaryWriter(f"runs/CIFAR10_tensorboard")

In [17]:
# TRAINING
if load_model:
    load_checkpoint(torch.load('my_checkpoint.pt'))

step = 0
for epoch in range(num_epochs):
    losses = []
    if epoch % 3 == 0:
        checkpoint = {'state_dict': model.state_dict(), 'optimizer':optimizer.state_dict()} # state dict is a python object that stores model's ( it can store optimizers parameters as well) parameters like conv1-bias, weight; conv2-bias weight and fc-bias weights etc.
        save_checkpoint(checkpoint)

    for batch_idx, (data, targets) in enumerate(train_loader): # data = img tensor, targets = labels tensor in related batch.
        data = data.to(device=device)
        targets = targets.to(device=device)

        # forward
        scores = model(data)
        loss = criterion(scores, targets)
        losses.append(loss.item())
        
        _, predictions = scores.max(1)
        num_correct = (predictions == targets).sum()
        running_train_acc = float(num_correct) / float(data.shape[0])

        # backward
        optimizer.zero_grad() # set gradient putput to zero for re-calculate gradient in every loop. We DO NOT want get interacted by previous gradient value.
        loss.backward() # compute gradient for every parameter

        # gradient descent or adam step 
        optimizer.step() # performs parameter update based on current computed gradient.
        writer.add_scalar("Trainin Loss :", loss, global_step=step)
        writer.add_scalar("Trainin ACC :", running_train_acc, global_step=step)
        step += 1

    mean_loss = sum(losses) / len(losses) # avarage of losses list.
    print(f"Loss at epoch {epoch} was {mean_loss:.5f}")

Loading Checkpoint
dict_keys(['state_dict', 'optimizer'])
Saving Checkpoint
Loss at epoch 0 was 0.00335
Loss at epoch 1 was 0.00195
Loss at epoch 2 was 0.00259
Saving Checkpoint
Loss at epoch 3 was 0.00156
Loss at epoch 4 was 0.00491
Loss at epoch 5 was 0.00142
Saving Checkpoint
Loss at epoch 6 was 0.00030
Loss at epoch 7 was 0.00324
Loss at epoch 8 was 0.00474
Saving Checkpoint
Loss at epoch 9 was 0.00072


In [8]:
def check_accuracy(loader, model):
    if loader.dataset.train:
        print("Checking acc on training data")
    else:
        print("Checking acc on test data")

    num_correct = 0
    num_samples = 0
    model.eval() # switch to evaluating ( inference ) mode

    with torch.no_grad(): # this context manager guarantees us that we are NOT CALCULATING gradient during this process.
        for x,y in loader:
            x = x.to(device=device)
            y = y.to(device=device)

            scores = model(x)  # scores output shape = 64x10
            value, predictions = scores.max(1) # we take max value of second dimension for getting which number have been predicted by NN.
            num_correct += (predictions == y).sum()  
            num_samples += predictions.size(0)   
        print(f"Got{num_correct} / {num_samples} with acc, {float(num_correct)/float(num_samples)*100:.2f}") # 95.78 etc.
    
    model.train()
    
check_accuracy(train_loader, model)
check_accuracy(test_loader, model)

Checking acc on training data
Got59639 / 60000 with acc, 99.40
Checking acc on test data
Got9870 / 10000 with acc, 98.70
